In [132]:
import pandas as pd
import numpy as np
import astropy.units as u
from astropy.coordinates import SkyCoord
from astropy.coordinates import match_coordinates_sky as match
from tqdm import tqdm

In [2]:
sncat = pd.read_csv('./The Open Supernova Catalog.csv', low_memory=False)

In [32]:
with open('./The Open Supernova Catalog.csv','r') as snfile:
    snlines = snfile.readlines()[:80245] # later than 041217

In [275]:
sn_name=[]#0
sn_ra=[]#4
sn_dec=[]#5
sn_z=[]#6
sn_type=[]#7
sn_host=[]#3
oddline=[] # len>12
oddname=[]
corline=[]
errline=[] # cannot read coordinate
errname=[]
for idx, line in tqdm(enumerate(snlines)):
    items = line.strip(' \n').split('","')
    name,ra,dec,z,typee,host = [x.strip(' "') for x in np.array(items)[np.array([0,4,5,6,7,3])]]
#    items = line.strip(' \n').split(',')
#    name,ra,dec,z,typee,host = [x.strip(' "') for x in np.array(items)[np.array([0,4,5,6,7,3])]]
    if len(items)>12:
       # print(name,idx,line)
        oddline.append(line)
        oddname.append(name)
        continue
    ras=ra.split(',')
    decs=dec.split(',')
    if 'GRB' in name: continue
    try:
        for i in range(len(ras)):
            c = SkyCoord(ras[i],decs[i],unit=(u.hourangle,u.deg))
            sn_name.append(name)
            sn_ra.append(ras[i])
            sn_dec.append(decs[i])
            sn_z.append(z)
            sn_type.append(typee)
            sn_host.append(host)
        corline.append(line)
    except:
        errline.append(line)

80245it [00:51, 1573.43it/s]


In [331]:
grbcat = pd.read_csv('./grb_table_1632362561.txt',low_memory=False, sep='\t')

# grb_name = np.array(grbcat['GRB'])
# grb_ra = np.array(grbcat['XRT RA (J2000)'])
# grb_dec = np.array(grbcat['XRT Dec (J2000)'])
# grb_ra_bat = np.array(grbcat['BAT RA (J2000)'])
# grb_dec_bat = np.array(grbcat['BAT Dec (J2000)'])
# grb_host = np.array(grbcat['Host Galaxy'])
# grb_z = np.array(grbcat['Redshift'])

grb_name,grb_ra,grb_dec,grb_host,grb_z,borx=[],[],[],[],[],[]

for idx in range(len(grbcat)):
    line=grbcat.loc[idx]
    name=line['GRB']
    ra_bat=line['BAT RA (J2000)']
    dec_bat=line['BAT Dec (J2000)']
    ra_xrt=line['XRT RA (J2000)']
    dec_xrt=line['XRT Dec (J2000)']
    host=line['Host Galaxy']
    z=line['Redshift']
    if ra_bat != '9999' and dec_bat != '9999':
        grb_name.append(name)
        grb_host.append(host)
        grb_z.append(z)
        if ra_xrt != '9999' and dec_xrt != '9999':
            grb_ra.append(ra_xrt)
            grb_dec.append(dec_xrt)
            borx.append('xrt')
        else:
            grb_ra.append((ra_bat*u.deg).to(u.hourangle))
            grb_dec.append(dec_bat)
            borx.append('bat')
            #print((dec_bat*u.deg))
    else:
        continue
    
# ### xrt coords
# xrt_idx = (grb_ra != '9999')
# grb_name = grb_name[xrt_idx]
# grb_ra = grb_ra[xrt_idx]
# grb_dec = grb_dec[xrt_idx]
# grb_host = grb_host[xrt_idx]
# grb_z = grb_z[valid_idx]

# ### bat coords
# xrt_idx = (grb_ra != '9999')
# grb_ra_bat = grb_ra_bat[valid_idx]
# grb_dec_bat = grb_dec_bat[valid_idx]

In [332]:
def sep2d(ra1,dec1,ra2,dec2):
    c1 = SkyCoord(ra1,dec1, unit=(u.hourangle, u.deg))
    c2 = SkyCoord(ra2,dec2, unit=(u.hourangle, u.deg))
    
    return (c1.separation(c2)*u.degree).value

In [333]:
grb_coo = SkyCoord(ra=grb_ra, dec=grb_dec, unit=(u.hourangle, u.deg))
sn_coo = SkyCoord(ra=sn_ra, dec=sn_dec, unit=(u.hourangle, u.deg))

In [334]:
idx,sep2d,dist3d=match(grb_coo, sn_coo)

In [335]:
threshold_xrt = float(10)/3600
threshold_bat = float(180)/3600

In [385]:
matched={}
for i,sep in enumerate(sep2d):
    gname=grb_name[i]
    gra=grb_ra[i]
    gdec=grb_dec[i]

    ghost=grb_host[i]
    if isinstance(ghost,str):
        ghost=ghost.replace(',',';')
        #print(ghost)
    gz=grb_z[i]
    if borx[i] == 'xrt':
        thres = threshold_xrt
    else:
        thres = threshold_bat
    
    sname=sn_name[idx[i]]
    sra,sdec=sn_ra[idx[i]],sn_dec[idx[i]]
    sz, stype, shost = sn_z[idx[i]], sn_type[idx[i]].replace(',',';'), sn_host[idx[i]].replace(',',';')
    if sep.value < thres:
        matched['{}-{}'.format(gname, sname)]=list(map(str,[sep.value * 3600,gra,gdec,ghost,gz, sra,sdec,stype,shost,sz]))

In [386]:
with open('grb-sn.csv','w') as gsfile:
    gsfile.write('grb-sn,sep(arcsec),gra,gdec,ghost,gz,sra,sdec,stype,shost,s\n')
    for item in matched.items():
        #print(','.join(item[1]))
        gsfile.write('{},{}\n'.format(item[0],','.join(item[1])))

## test

In [174]:
grb_coo[:2],sn_coo[:2]

(<SkyCoord (ICRS): (ra, dec) in deg
     [( 80.254375,  1.31169444), (129.892125, 79.22119444)]>,
 <SkyCoord (ICRS): (ra, dec) in deg
     [(16.1158   , 39.44238611), (35.7978875, 14.38173611)]>)

In [176]:
(grb_coo[0].separation(sn_coo[1])*u.degree).value

45.81967285098611

In [370]:
'lasg,asdf,as'.replace(',','.')

'lasg.asdf.as'